In [1]:
import pandas as pd
import sqlalchemy
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool  #type: ignore
from sqlalchemy import create_engine, text
from sqlalchemy.engine.base import Engine

* A função create_engine do SQLAlchemy para nos conectarmos ao banco de dados, que é um arquivo local chamado `sales.db`.
* Estabelecer a ponte de comunicação entre nosso código Python e o banco de dados. A variável db_engine agora representa essa conexão ativa.

In [2]:
db_engine = create_engine("sqlite:///sales.db")

As células abaixo 3 a 6 exploram o banco de dados.

O Código: Primeiro, criamos um objeto `inspector` para "olhar dentro" do banco de dados. Em seguida, usamos esse `inspetor` para listar os nomes das **tabelas** e depois para obter os detalhes das colunas de uma tabela específica, a **"sales"**. Por fim, extraímos apenas os nomes das colunas para uma lista.

O Objetivo: **Antes de fazer perguntas, precisamos saber o que há no banco de dados. Esta é a fase de reconhecimento. Estamos descobrindo quais tabelas existem e qual é a estrutura delas.**

In [3]:
inspector = sqlalchemy.inspect(db_engine)

In [4]:
inspector.get_table_names()

['sales']

In [5]:
table_name = "sales"
inspector.get_columns(table_name)

[{'name': 'id',
  'type': INTEGER(),
  'nullable': False,
  'default': None,
  'primary_key': 1},
 {'name': 'transaction_date',
  'type': DATE(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'model',
  'type': VARCHAR(length=50),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'price',
  'type': FLOAT(),
  'nullable': False,
  'default': None,
  'primary_key': 0},
 {'name': 'quantity',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'customer_id',
  'type': INTEGER(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

In [6]:
schema = inspector.get_columns(table_name)
column_names = [column["name"] for column in schema]
column_names

['id', 'transaction_date', 'model', 'price', 'quantity', 'customer_id']

As células 7-10: Executando uma query de teste

**O Código**: Construímos uma string SQL para selecionar as 10 primeiras linhas da tabela "sales". Usamos o db_engine para executar essa query e guardar o resultado na variável answer. A célula 9 mostra o resultado "cru" (uma lista de tuplas). A célula 10 usa o Pandas para mostrar esse mesmo resultado de forma organizada.

**O Objetivo**: Validar nossa conexão e entender o formato dos dados. A comparação entre a saída da célula 9 e 10 demonstra claramente o valor do Pandas para a legibilidade.

In [7]:
sql = f"SELECT * FROM {table_name} LIMIT 10"

In [8]:
with db_engine.begin() as connection:
    answer = connection.execute(text(sql)).fetchall()

In [9]:
answer

[(1, '2024-05-22', 'Dell XPS 15', 1308.04, 4, 1037),
 (2, '2024-06-07', 'Dell Inspiron 15', 2399.23, 4, 1044),
 (3, '2024-12-07', 'Dell XPS 13', 2481.61, 2, 1013),
 (4, '2024-06-21', 'Dell G5 15', 505.98, 4, 1054),
 (5, '2024-01-04', 'Dell Inspiron 14', 1429.78, 1, 1097),
 (6, '2024-04-23', 'Dell Inspiron 14', 1141.06, 2, 1074),
 (7, '2024-06-18', 'Dell Latitude 7310', 1740.03, 2, 1094),
 (8, '2024-03-12', 'Dell XPS 15', 521.49, 3, 1019),
 (9, '2024-02-10', 'Dell Latitude 7410', 545.36, 4, 1003),
 (10, '2024-12-17', 'Dell Inspiron 14', 668.95, 3, 1010)]

In [10]:
pd.DataFrame(answer, columns=column_names)

,id,transaction_date,model,price,quantity,customer_id
0,1,2024-05-22,Dell XPS 15,1308.04,4,1037
1,2,2024-06-07,Dell Inspiron 15,2399.23,4,1044
2,3,2024-12-07,Dell XPS 13,2481.61,2,1013
3,4,2024-06-21,Dell G5 15,505.98,4,1054
4,5,2024-01-04,Dell Inspiron 14,1429.78,1,1097
5,6,2024-04-23,Dell Inspiron 14,1141.06,2,1074
6,7,2024-06-18,Dell Latitude 7310,1740.03,2,1094
7,8,2024-03-12,Dell XPS 15,521.49,3,1019
8,9,2024-02-10,Dell Latitude 7410,545.36,4,1003
9,10,2024-12-17,Dell Inspiron 14,668.95,3,1010


**Tools**

# Célula 11
@tool
def list_tables_tool(config: RunnableConfig) -> list[str]:
    #...
# Célula 12
@tool
def get_table_schema_tool(table_name: str, config: RunnableConfig) -> list[str]:
    #...
# Célula 13
@tool
def execute_sql_tool(query: str, config: RunnableConfig) -> int:
    #...

* O Código: Cada célula define uma função Python que encapsula uma das ações que fizemos manualmente antes (listar tabelas, obter esquema, executar SQL).

* O Objetivo: Transformar nossa lógica de exploração em ferramentas reutilizáveis para o agente.

Conceitos-Chave:

`@tool`: **O decorador que diz ao LangChain: "Ei, esta função é uma ferramenta que um LLM pode usar!"**.

Docstring """...""": É a descrição que o LLM vai ler para entender o que a ferramenta faz. Essencial!.

`config: RunnableConfig`: Este é o mecanismo moderno para passar dependências (como o db_engine) para a ferramenta de forma segura e sem usar variáveis globais.

In [11]:
@tool
def list_tables_tool(config: RunnableConfig) -> list[str]:
    """List all tables in database
    """
    db_engine: Engine = config.get("configurable", {}).get("db_engine")
    inspector = sqlalchemy.inspect(db_engine)

    return inspector.get_table_names()

In [12]:
@tool
def get_table_schema_tool(table_name: str, config: RunnableConfig) -> list[str]:
    """Get schema information about a table. Returns a list of dictionaries.
    - name is the column name
    - type is the column type
    - nullable is whether the column is nullable or not
    - default is the default value of the column
    - primary_key is whether the column is a primary key or not

    Args:
        table_name (str): Table name

    """
    db_engine: Engine = config.get("configurable", {}).get("db_engine")
    inspector = sqlalchemy.inspect(db_engine)

    return inspector.get_columns(table_name)


In [13]:
@tool
def execute_sql_tool(query: str, config: RunnableConfig) -> int:
    """Execute SQL query and return result.
    This will automatically connect to the database and execute the query.
    However, if the query is not valid, an error will be raised

    Args:
        query (str): SQL query

    """
    db_engine: Engine = config.get("configurable", {}).get("db_engine")
    with db_engine.begin() as connection:
        answer = connection.execute(text(query)).fetchall()

    return answer

Testando as ferramentas 14 a 18

* **O Código**: Recriamos o db_engine e criamos um dicionário config para "empacotar" nossa conexão. Em seguida, chamamos cada uma das nossas ferramentas usando o método .invoke(), passando os argumentos necessários e o config.

* **O Objetivo**: Verificar se as ferramentas que criamos estão funcionando como esperado antes de entregá-las a um agente. É a fase de teste unitário das nossas ferramentas. Vemos como o config é usado para dar à ferramenta o acesso ao banco de dados no momento exato da sua execução.

In [14]:
db_engine = create_engine("sqlite:///sales.db")

In [15]:
config = {"configurable": {"db_engine": db_engine}}

In [16]:
tables = list_tables_tool.invoke({}, config)
tables

['sales']

In [17]:
schemas = {
    table: get_table_schema_tool.invoke({
            "table_name": table
        }, config)
    for table in tables
}
schemas

{'sales': [{'name': 'id',
   'type': INTEGER(),
   'nullable': False,
   'default': None,
   'primary_key': 1},
  {'name': 'transaction_date',
   'type': DATE(),
   'nullable': False,
   'default': None,
   'primary_key': 0},
  {'name': 'model',
   'type': VARCHAR(length=50),
   'nullable': False,
   'default': None,
   'primary_key': 0},
  {'name': 'price',
   'type': FLOAT(),
   'nullable': False,
   'default': None,
   'primary_key': 0},
  {'name': 'quantity',
   'type': INTEGER(),
   'nullable': True,
   'default': None,
   'primary_key': 0},
  {'name': 'customer_id',
   'type': INTEGER(),
   'nullable': True,
   'default': None,
   'primary_key': 0}]}

In [18]:
sql = f"SELECT * FROM {tables[0]} LIMIT 10"
result = execute_sql_tool.invoke({"query": sql}, config)
result

[(1, '2024-05-22', 'Dell XPS 15', 1308.04, 4, 1037),
 (2, '2024-06-07', 'Dell Inspiron 15', 2399.23, 4, 1044),
 (3, '2024-12-07', 'Dell XPS 13', 2481.61, 2, 1013),
 (4, '2024-06-21', 'Dell G5 15', 505.98, 4, 1054),
 (5, '2024-01-04', 'Dell Inspiron 14', 1429.78, 1, 1097),
 (6, '2024-04-23', 'Dell Inspiron 14', 1141.06, 2, 1074),
 (7, '2024-06-18', 'Dell Latitude 7310', 1740.03, 2, 1094),
 (8, '2024-03-12', 'Dell XPS 15', 521.49, 3, 1019),
 (9, '2024-02-10', 'Dell Latitude 7410', 545.36, 4, 1003),
 (10, '2024-12-17', 'Dell Inspiron 14', 668.95, 3, 1010)]